In [1]:
import pandas as pd

- df => icustays
- df2 => admissions
- df3 => patients

In [14]:
df = pd.read_csv('../data/icustays_ibd.csv')

In [15]:
df['subject_id'].unique().shape

(683,)

In [17]:
df2 = pd.read_csv('../data/admissions_ibd.csv')

def parse_race(race):
    if 'WHITE' in race:
        return 'WHITE'
    elif 'BLACK' in race:
        return 'BLACK'
    elif 'HISPANIC' in race or 'LATINO' in race:
        return 'HISPANIC/LATINO'
    elif 'ASIAN' in race:
        return 'ASIAN'
    else:
        return 'OTHER'

df2['race'] = df2['race'].apply(parse_race)

df2['admittime'] = pd.to_datetime(df2['admittime']) # 入院时间
df2['dischtime'] = pd.to_datetime(df2['dischtime']) # 出院时间
df2['deathtime'] = pd.to_datetime(df2['deathtime']) # 死亡时间
df2['edregtime'] = pd.to_datetime(df2['edregtime']) # 入急诊时间
df2['edouttime'] = pd.to_datetime(df2['edouttime']) # 出急诊时间

df2

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10098672,21229395,2142-05-16 04:04:00,2142-05-23 14:15:00,NaT,EW EMER.,P30SZF,EMERGENCY ROOM,HOME HEALTH CARE,Other,ENGLISH,DIVORCED,WHITE,2142-05-15 15:49:00,2142-05-16 05:45:00,0
1,10098672,21259834,2141-04-13 17:30:00,2141-04-17 19:17:00,NaT,EW EMER.,P77SO2,EMERGENCY ROOM,HOME HEALTH CARE,Other,ENGLISH,DIVORCED,WHITE,2141-04-13 12:17:00,2141-04-13 18:47:00,0
2,10098672,21921527,2141-06-29 22:55:00,2141-07-03 16:52:00,NaT,URGENT,P816G0,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,DIVORCED,WHITE,NaT,NaT,0
3,10098672,24843492,2142-05-06 07:15:00,2142-05-11 13:39:00,NaT,SURGICAL SAME DAY ADMISSION,P857XC,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,ENGLISH,DIVORCED,WHITE,NaT,NaT,0
4,10098672,26570321,2142-01-20 23:24:00,2142-01-28 18:47:00,NaT,EW EMER.,P95PIA,EMERGENCY ROOM,HOME HEALTH CARE,Other,ENGLISH,DIVORCED,WHITE,2142-01-20 19:54:00,2142-01-21 01:07:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10605,16308779,27539941,2138-01-02 04:15:00,2138-01-04 16:35:00,NaT,EU OBSERVATION,P886Y0,EMERGENCY ROOM,NaN,Medicare,ENGLISH,WIDOWED,WHITE,2138-01-01 20:23:00,2138-01-02 04:51:00,0
10606,10556676,20224885,2167-04-16 15:33:00,2167-04-17 09:30:00,NaT,EU OBSERVATION,P76VMW,EMERGENCY ROOM,NaN,Other,ENGLISH,MARRIED,WHITE,2167-04-16 07:16:00,2167-04-17 09:30:00,0
10607,10556676,22783191,2164-02-12 04:35:00,2164-02-14 12:45:00,NaT,EW EMER.,P534S8,EMERGENCY ROOM,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,2164-02-11 17:41:00,2164-02-12 05:11:00,0
10608,10556676,25577156,2163-10-31 21:05:00,2163-12-19 18:00:00,NaT,EW EMER.,P534S8,EMERGENCY ROOM,CHRONIC/LONG TERM ACUTE CARE,Other,ENGLISH,MARRIED,WHITE,2163-10-31 18:28:00,2163-10-31 23:48:00,0


In [18]:
hadm_ids_dead = df2[df2['deathtime'] == df2['dischtime']]['hadm_id'] # admission 中死亡的 hadm_id

In [23]:
hadm_ids_dead.shape, hadm_ids_dead.unique().shape # 判断 103 是不是 unique

((103,), (103,))

In [24]:
df_dead = df[df['hadm_id'].isin(hadm_ids_dead)] # icu 事件就是死亡事件的记录
df_alive = df[~df['hadm_id'].isin(hadm_ids_dead)] # icu 事件不是死亡事件的记录

In [29]:
df['hadm_id'].unique().shape, df_dead['hadm_id'].unique().shape, df_alive['hadm_id'].unique().shape

((1161,), (84,), (1077,))

In [30]:
# admissions 中有死亡事件记录
# icustats 中有死亡事件记录
# 推断: 死在 ICU
df2[df2['deathtime'] == df2['dischtime']][df2[df2['deathtime'] == df2['dischtime']]['hadm_id'].isin(df_dead['hadm_id'])].shape

(84, 16)

In [31]:
# admissions 中有死亡事件记录 
# icustats 没中有死亡事件记录
# 推断: 不是死在 ICU
df2[df2['deathtime'] == df2['dischtime']][~df2[df2['deathtime'] == df2['dischtime']]['hadm_id'].isin(df_dead['hadm_id'])].shape

(19, 16)

In [129]:
df3 = pd.read_csv('../data/patients_ibd.csv')
df3['anchor_year'] = df3['anchor_year'].apply(lambda x: pd.to_datetime('{}-01-01'.format(x)))
df3['dod'] = pd.to_datetime(df3['dod'])
df3.head()

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10098672,M,61,2140-01-01,2011 - 2013,NaT
1,10303503,F,23,2144-01-01,2008 - 2010,NaT
2,10312715,M,39,2176-01-01,2008 - 2010,NaT
3,10318500,F,46,2194-01-01,2011 - 2013,NaT
4,10410021,M,49,2135-01-01,2011 - 2013,NaT


In [130]:
# df2_dead = df2[df2['deathtime'] == df2['dischtime']] # admission 事件就是死亡事件
# df2_alive = df2[~(df2['deathtime'] == df2['dischtime'])] # admission 事件不是死亡事件

df2_dead = df2[~df2['deathtime'].isna()] # admission 事件就是死亡事件
df2_alive = df2[df2['deathtime'].isna()] # admission 事件不是死亡事件

df2.shape, df2_dead.shape, df2_alive.shape

((10610, 16), (111, 16), (10499, 16))

In [131]:
_ = df2[['subject_id', 'deathtime']].groupby('subject_id').last().reset_index()
__ = _[
    _['subject_id'].isin(
        df3[~df3['dod'].isna()]['subject_id']
    )
]

___ = __[~__['subject_id'].isin(df2_dead['subject_id'])]

___['subject_id'].shape # 269 dead not in hospital

(269,)

In [156]:
tmp = df3[~df3['dod'].isna()]
tmp['d_age'] = (tmp['dod'] - tmp['anchor_year']).dt.days / 365 + tmp['anchor_age']

tmp['d_age'].mean(), tmp['d_age'].std(), tmp['subject_id'].unique().shape

C:\Users\sitdo\AppData\Local\Temp\ipykernel_10772\1160301568.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['d_age'] = (tmp['dod'] - tmp['anchor_year']).dt.days / 365 + tmp['anchor_age']


(72.73695746214852, 13.959149622963654, (380,))

In [132]:
tmp = df2_dead.merge(df3, on='subject_id')
tmp['d_age'] = (tmp['deathtime'] - tmp['anchor_year']).dt.days / 365 + tmp['anchor_age']

tmp['d_age'].mean(), tmp['d_age'].std(), tmp['subject_id'].unique().shape

(70.88577070220906, 13.68585808474427, (111,))

In [149]:
tmp = df3[df3['subject_id'].isin(___['subject_id'])]
# tmp['d_age'] = (tmp['dod'] - tmp['anchor_year']).dt.days / 365 + tmp['anchor_age']

# tmp['d_age'].mean(), tmp['d_age'].std(), tmp['subject_id'].unique().shape

tmp['d_age'] = (tmp['dod'] - tmp['anchor_year']).dt.days / 365 + tmp['anchor_age']
tmp['d_age'].mean(), tmp['d_age'].std(), tmp['subject_id'].unique().shape

C:\Users\sitdo\AppData\Local\Temp\ipykernel_10772\1008880505.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['d_age'] = (tmp['dod'] - tmp['anchor_year']).dt.days / 365 + tmp['anchor_age']


(73.50083006569231, 14.024297022139857, (269,))

> admissions 和 icustays 两张表格

admissions 中一共有 103 个病人有死亡记录

- 其中 有 84 个病人是在 icustays 中有记录，暂且认为是死在了 ICU

- 另外 有 19 个病人不在 icustays 中有记录，暂且认为是死在了其他地方

In [79]:
# 84 个死在 ICU 的平均死亡年龄
tmp = df2_dead[df2_dead['subject_id'].isin(df_dead['subject_id'])].merge(df3, on='subject_id')
tmp['d_age'] = (tmp['deathtime'] - tmp['anchor_year']).dt.days / 365 + tmp['anchor_age']

tmp['d_age'].mean(), tmp['d_age'].std(), tmp['subject_id'].unique().shape

(70.99500978473581, 13.239634625711627, (84,))

In [80]:
# 19 个不是死在 ICU 的平均死亡年龄
tmp = df2_dead[~df2_dead['subject_id'].isin(df_dead['subject_id'])].merge(df3, on='subject_id')
tmp['d_age'] = (tmp['deathtime'] - tmp['anchor_year']).dt.days / 365 + tmp['anchor_age']

tmp['d_age'].mean(), tmp['d_age'].std(), tmp['subject_id'].unique().shape

(70.54591577879249, 15.254607836065965, (27,))

**惊奇的发现，死在了 ICU 的病人和 不是死在了 ICU 的病人的平均死亡年龄非常接近！**

分别是 70.99 岁和 70.85 岁

> Range: 683 个来过 ICU 的病人

| | 没死 | 死了 | 死在 ICU | 死在别的地方 |
|-| -- | --- | -------- | ------------- |
|人数| 588 | 95 |  84 | 11 |
| AVG LOS | 5.94 | 8.47   | 9.24 | 5.88 |

In [171]:
stats = df.groupby('subject_id').agg({
    'los': ['count', 'sum']
}) # 用 df 标志进过 ICU

stats.columns = ['count', 'sum']
stats = stats.reset_index()

In [175]:
stats['dead'] = stats['subject_id'].isin(df2_dead['subject_id'])
stats['dead_icu'] = stats['subject_id'].isin(df_dead['subject_id'])
stats.head()

,subject_id,count,sum,dead,dead_icu
0,10024331,1,4.10571,False,False
1,10025647,1,1.96810,False,False
2,10037975,1,4.87824,True,True
3,10048262,1,0.44588,False,False
4,10056223,1,5.04106,False,False


In [179]:
(
    stats[stats['dead']]['count'].shape,
    stats[~stats['dead']]['count'].shape,
    stats[stats['dead_icu']]['count'].shape,
    stats[~stats['dead_icu']]['count'].shape,
)

((103,), (580,), (84,), (599,))

In [180]:
(
    stats[stats['dead']]['count'].mean(),
    stats[~stats['dead']]['count'].mean(),
    stats[stats['dead_icu']]['count'].mean(),
    stats[~stats['dead_icu']]['count'].mean(),
)

(2.203883495145631, 1.8862068965517242, 2.3333333333333335, 1.8781302170283807)

In [182]:
(
    stats[stats['dead']]['count'].std(),
    stats[~stats['dead']]['count'].std(),
    stats[stats['dead_icu']]['count'].std(),
    stats[~stats['dead_icu']]['count'].std(),
)

(2.1844334451957157, 2.125789360724616, 2.3711790231526924, 2.0971805830044508)

In [181]:
(
    stats[stats['dead']]['sum'].mean(),
    stats[~stats['dead']]['sum'].mean(),
    stats[stats['dead_icu']]['sum'].mean(),
    stats[~stats['dead_icu']]['sum'].mean(),
)

(8.736096291262136, 5.865734888793103, 9.240651583333337, 5.886025743739566)

In [183]:
(
    stats[stats['dead']]['sum'].std(),
    stats[~stats['dead']]['sum'].std(),
    stats[stats['dead_icu']]['sum'].std(),
    stats[~stats['dead_icu']]['sum'].std(),
)

(11.309263738215272, 8.9044526510451, 11.841929471773277, 8.884867451581764)

In [202]:
tmp = df2.groupby('subject_id').first().merge(df3, on='subject_id')

tmp['first_age'] = (tmp['admittime'] - tmp['anchor_year']).dt.days / 365 + tmp['anchor_age']


(
    (tmp[tmp['subject_id'].isin(stats['subject_id'])]['first_age'].mean(), tmp[tmp['subject_id'].isin(stats['subject_id'])]['first_age'].std(), tmp[tmp['subject_id'].isin(stats['subject_id'])]['first_age'].shape),
    (tmp[~tmp['subject_id'].isin(stats['subject_id'])]['first_age'].mean(), tmp[~tmp['subject_id'].isin(stats['subject_id'])]['first_age'].std(), tmp[~tmp['subject_id'].isin(stats['subject_id'])]['first_age'].shape),
)


((62.32124992478789, 16.587436666014305, (683,)),
 (49.819811663585654, 18.48115822618673, (1734,)))

In [213]:
(~tmp[~tmp['subject_id'].isin(stats['subject_id'])]['dod'].isna()).sum(), tmp[~tmp['subject_id'].isin(stats['subject_id'])]['dod'].isna().sum()

(125, 1609)

In [214]:
for col in ['language', 'marital_status', 'race', 'insurance']:
    print('- total')
    print(df2[col].value_counts())
    print(df2[col].value_counts() / df2[col].value_counts().sum())
    print('- dead')
    # print(df2_dead[col].value_counts())
    # print(df2_dead[col].value_counts() / df2_dead[col].value_counts().sum())
    print()

- total
language
ENGLISH    10252
?            358
Name: count, dtype: int64
language
ENGLISH    0.966258
?          0.033742
Name: count, dtype: float64
- dead

- total
marital_status
MARRIED     4773
SINGLE      4167
WIDOWED      831
DIVORCED     802
Name: count, dtype: int64
marital_status
MARRIED     0.451433
SINGLE      0.394117
WIDOWED     0.078596
DIVORCED    0.075854
Name: count, dtype: float64
- dead

- total
race
WHITE              8776
BLACK               931
OTHER               495
HISPANIC/LATINO     329
ASIAN                79
Name: count, dtype: int64
race
WHITE              0.827144
BLACK              0.087747
OTHER              0.046654
HISPANIC/LATINO    0.031008
ASIAN              0.007446
Name: count, dtype: float64
- dead

- total
insurance
Other       6255
Medicare    3673
Medicaid     682
Name: count, dtype: int64
insurance
Other       0.589538
Medicare    0.346183
Medicaid    0.064279
Name: count, dtype: float64
- dead



In [166]:
(
    (df2[['subject_id', 'deathtime']].groupby('subject_id').last().reset_index()['deathtime'].isna()).sum(),
    (~df2[['subject_id', 'deathtime']].groupby('subject_id').last().reset_index()['deathtime'].isna()).sum()
)

(2306, 111)

In [167]:
df2[df2['deathtime'].isna()]['subject_id'].unique().shape

(2401,)

In [168]:
(df3['dod'].isna()).sum()

2037

In [169]:
_ = df2[~df2['deathtime'].isna()]

(_['deathtime'] == _['dischtime']).sum(), (_['deathtime'] != _['dischtime']).sum()

(103, 8)

In [170]:
_[_['deathtime'] != _['dischtime']]

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
5077,12486920,20344681,2140-08-06 11:33:00,2140-08-19 04:30:00,2140-08-19 03:00:00,DIRECT EMER.,P86FCY,CLINIC REFERRAL,DIED,Medicare,ENGLISH,SINGLE,HISPANIC/LATINO,NaT,NaT,1
5522,17995700,23971774,2149-06-16 16:02:00,2149-06-19 04:21:00,2149-06-19 23:45:00,URGENT,P17MK7,TRANSFER FROM HOSPITAL,DIED,Medicare,ENGLISH,MARRIED,WHITE,NaT,NaT,1
6001,13358539,26097204,2138-03-04 22:11:00,2138-03-17 00:00:00,2138-03-17 22:45:00,EW EMER.,P27PNT,EMERGENCY ROOM,DIED,Other,ENGLISH,SINGLE,OTHER,2138-03-04 14:33:00,2138-03-05 00:52:00,1
6467,19940147,23739599,2129-12-28 16:21:00,2130-01-15 00:31:00,2130-01-15 19:31:00,DIRECT EMER.,P55Z20,PHYSICIAN REFERRAL,DIED,Medicare,ENGLISH,MARRIED,WHITE,NaT,NaT,1
7833,15071757,29776979,2132-06-27 22:58:00,2132-06-29 00:00:00,2132-06-29 20:00:00,EW EMER.,P3529J,EMERGENCY ROOM,DIED,Other,ENGLISH,MARRIED,WHITE,2132-06-27 15:36:00,2132-06-27 23:46:00,1
8044,10701573,24314052,2181-11-24 19:30:00,2181-12-09 00:00:00,2181-12-09 17:15:00,EW EMER.,P38TI6,EMERGENCY ROOM,DIED,Medicaid,ENGLISH,WIDOWED,BLACK,2181-11-24 15:30:00,2181-11-24 20:59:00,1
9303,15342986,24280081,2134-02-13 13:05:00,2134-02-17 01:40:00,2134-02-17 22:30:00,EW EMER.,P79SJ2,EMERGENCY ROOM,DIED,Medicare,ENGLISH,MARRIED,WHITE,2134-02-13 11:44:00,2134-02-13 15:20:00,1
10090,11735934,20948812,2138-07-10 16:18:00,2138-07-17 00:48:00,2138-07-17 21:50:00,EW EMER.,P30KEH,EMERGENCY ROOM,DIED,Medicare,ENGLISH,WIDOWED,WHITE,2138-07-10 14:06:00,2138-07-10 17:20:00,1
